In [30]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [31]:
city_data_df = pd.read_csv('weather_data/WeatherPy_Database.csv')
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Visby,SE,57.64,18.30,61.0,87,85,21.92,overcast clouds
1,1,Leshukonskoye,RU,64.90,45.76,37.2,79,0,4.43,clear sky
2,2,Lata,PT,40.16,-8.33,61.0,91,73,5.99,broken clouds
3,3,Ambon,ID,-3.70,128.18,87.8,66,40,4.70,scattered clouds
4,4,Hasaki,JP,35.73,140.83,81.0,73,75,8.05,broken clouds


In [32]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("what is then min temperature you would like to trip"))
max_temp = float(input("what is then max temperature you would like to trip"))

what is then min temperature you would like to trip60
what is then max temperature you would like to trip75


In [33]:
preferred_city_df = city_data_df.loc[(city_data_df['Max Temp']<= max_temp)&(city_data_df['Max Temp'] >= min_temp)]
preferred_city_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Visby,SE,57.64,18.30,61.00,87,85,21.92,overcast clouds
2,2,Lata,PT,40.16,-8.33,61.00,91,73,5.99,broken clouds
6,6,Hongjiang,CN,27.11,110.00,63.61,88,100,5.39,light rain
16,16,Pisco,PE,-13.70,-76.22,61.00,82,14,1.12,smoke
18,18,Jabinyanah,TN,35.03,10.91,69.80,64,40,4.70,scattered clouds


In [34]:
preferred_city_df.isnull().count()

City ID                175
City                   175
Country                175
Lat                    175
Lng                    175
Max Temp               175
Humidity               175
Cloudiness             175
Wind Speed             175
Current Description    175
dtype: int64

In [35]:
hotel_df = preferred_city_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Visby,SE,61.00,overcast clouds,57.64,18.30,
2,Lata,PT,61.00,broken clouds,40.16,-8.33,
6,Hongjiang,CN,63.61,light rain,27.11,110.00,
16,Pisco,PE,61.00,smoke,-13.70,-76.22,
18,Jabinyanah,TN,69.80,scattered clouds,35.03,10.91,
21,Jining,CN,62.65,overcast clouds,35.41,116.58,
26,Rikitea,PF,73.62,clear sky,-23.12,-134.97,
29,Ngaoundere,CM,60.80,scattered clouds,7.32,13.58,
30,Joshimath,IN,72.95,clear sky,30.57,79.57,
50,Huarmey,PE,61.90,broken clouds,-10.07,-78.15,


In [36]:
from config import g_key
params = {"radius": 5000,"types": "lodging","key": g_key,"location": "-10.17, 123.58"}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params['location']=(f"{lat},{lng}")
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [37]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Visby,SE,61.00,overcast clouds,57.64,18.30,Best Western Solhem Hotel Visby
2,Lata,PT,61.00,broken clouds,40.16,-8.33,Casa Maquia
6,Hongjiang,CN,63.61,light rain,27.11,110.00,洪江大酒店
16,Pisco,PE,61.00,smoke,-13.70,-76.22,Hostal San Isidro Oficial
18,Jabinyanah,TN,69.80,scattered clouds,35.03,10.91,maison belmabrouk
...,...,...,...,...,...,...,...
683,Mizan Teferi,ET,66.65,light rain,6.98,35.58,Hotel Salayish | ሆቴል ሳላይሽ
684,Chitral,PK,70.29,clear sky,35.85,71.79,PTDC Motel Chitral
688,Itabira,BR,65.79,few clouds,-19.62,-43.23,Premium Executive Hotel
689,Marsabit,KE,68.94,scattered clouds,2.33,37.99,Chicho Hotel


In [38]:
output_data_file = "weather_data/vocation.csv"
hotel_df.to_csv(output_data_file, index_label = "City ID")

In [39]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))